<a href="https://colab.research.google.com/github/EkaFitriRamadani/PengantarDeepLearning2022/blob/main/DeepLearningComputationD2l_12March22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Layers and Blocks

In [ ]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.00810969,  0.18566215,  0.26851723,  0.10982366,  0.06855363,
        -0.14153041, -0.02970569, -0.15734035, -0.12218191, -0.22810626],
       [-0.0271264 ,  0.13448231,  0.36666396,  0.02783715,  0.01078771,
        -0.27087727,  0.14821039, -0.18813287,  0.00512167, -0.3762917 ]],
      dtype=float32)>

## 1.1 Custom Block

In [ ]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Model` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def call(self, X):
        return self.out(self.hidden((X)))

In [ ]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.2241644 , -0.04072355, -0.11540591, -0.4091962 , -0.09980948,
        -0.05294572, -0.12228737, -0.07768904, -0.03691309,  0.12446226],
       [ 0.16143873, -0.05556329,  0.13090585, -0.3097669 ,  0.00105227,
         0.12259   , -0.14585601, -0.05665173,  0.03180757,  0.0386494 ]],
      dtype=float32)>

## 1.2 The Sequential Block

In [ ]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [ ]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.01114691, -0.03349262, -0.0199593 , -0.34126845,  0.23891993,
         0.00144508, -0.05225813, -0.01497521,  0.53210765,  0.05355164],
       [ 0.09835127,  0.06520436, -0.03226636, -0.27802676,  0.32658634,
        -0.01826696, -0.18004668, -0.07918334,  0.5231236 , -0.00819121]],
      dtype=float32)>

## 1.3 Executing Code in The Forward Propagation Function

In [ ]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with `tf.constant` are not updated
        # during training (i.e., constant parameters)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the `relu` and
        # `matmul` functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [ ]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.58108795>

In [ ]:
# We can mix and match various ways of assembling blocks together. In the following example, we nest blocks in some creative ways.
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.60099155>

# 2. Parameter Management

In [1]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.87241846],
       [0.8839122 ]], dtype=float32)>

## 2.1 Parameter Access

In [2]:
print(net.layers[2].weights)

[<tf.Variable 'dense_1/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.73814666],
       [ 0.64098537],
       [ 0.45175457],
       [-0.12581688]], dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


### 2.1.1 Targeted parameters

In [3]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


2.1.2 All Parameters at Once

In [4]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.236561  ,  0.8038214 , -0.19574326,  0.58632237],
       [ 0.49491316, -0.3367647 , -0.11646926, -0.19637549],
       [-0.39434665,  0.15174598,  0.31523043,  0.66565996],
       [ 0.44136506,  0.5503294 ,  0.06087178, -0.01026827]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.236561  ,  0.8038214 , -0.19574326,  0.58632237],
       [ 0.49491316, -0.3367647 , -0.11646926, -0.19637549],
       [-0.39434665,  0.15174598,  0.31523043,  0.66565996],
       [ 0.44136506,  0.5503294 ,  0.06087178, -0.01026827]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 0.73814666],
       [ 0.64098537],
       [ 0.45175457],
       [-0.12581688]], dtype=float32), array([0.], dtype=float32)]


In [5]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

### 2.1.3 Collecting Parameters from Nested Blocks

In [6]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.06987841],
       [0.05362102]], dtype=float32)>

In [7]:
print(rgnet.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (2, 4)                    80        
                                                                 
 dense_6 (Dense)             (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_3/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

## 2.2 Parameters Initialization

### 2.2.1 Built-in Initialization

In [9]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_7/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.00745601, -0.00337034, -0.01078232,  0.01890916],
        [-0.0122731 , -0.0090603 ,  0.01651362, -0.01308391],
        [ 0.00546503,  0.00411272,  0.01395592,  0.00995624],
        [-0.00064154,  0.01731787,  0.0048789 ,  0.00113453]],
       dtype=float32)>,
 <tf.Variable 'dense_7/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [10]:
# We can also initialize all the parameters to a given constant value (say, 1).
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [11]:
# We can also apply different initializers for certain blocks. 
# For example, below we initialize the first layer with the Xavier 
# initializer and initialize the second layer to a constant value of 42.
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.75909615, -0.84215194,  0.29570872,  0.04361117],
       [ 0.08411604,  0.31379908,  0.44000787,  0.4311183 ],
       [ 0.52803344, -0.71835995,  0.08313406,  0.44862133],
       [ 0.20678586, -0.02807939,  0.85635155, -0.21430486]],
      dtype=float32)>
<tf.Variable 'dense_12/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


### 2.2.2 Custom Initialization

In [12]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-9.207327 , -9.32143  , -0.       ,  0.       ],
       [ 9.823133 ,  8.62447  , -5.1590204,  0.       ],
       [ 8.035023 ,  5.6136703, -8.221781 , -7.6365423],
       [-0.       , -6.7836547,  0.       , -0.       ]], dtype=float32)>


In [13]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       , -8.32143  ,  1.       ,  1.       ],
       [10.823133 ,  9.62447  , -4.1590204,  1.       ],
       [ 9.035023 ,  6.6136703, -7.221781 , -6.6365423],
       [ 1.       , -5.7836547,  1.       ,  1.       ]], dtype=float32)>

## 2.3 Tied parameters

In [14]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


# 3. Deffered Initialization

## 3.1 Instantiating a Network

In [15]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [16]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [17]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

# 4. Custom Layers

## 4.1 Layers without Parameters

In [18]:
import tensorflow as tf


class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [19]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [20]:
# We can now incorporate our layer as a component in constructing more complex models.
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [21]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=-9.313226e-10>

## 4.2 Layers with Parameters

In [22]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [23]:
# Next, we instantiate the MyDense class and access its model parameters.
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[-0.09186442,  0.03891678, -0.02555907],
        [-0.04568631,  0.02148853, -0.07752487],
        [-0.00801391, -0.03195686,  0.06860649],
        [ 0.1242008 , -0.02096754,  0.01969148],
        [ 0.0184412 , -0.02668449,  0.0953656 ]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [24]:
# We can directly carry out forward propagation calculations using custom layers.
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.        , 0.00032671, 0.05580046],
       [0.        , 0.        , 0.06021539]], dtype=float32)>

In [26]:
# net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.01396879],
       [0.00656588]], dtype=float32)>

# 5. File I/O

## 5.1 Loading and Saving Tensors

In [27]:
import numpy as np
import tensorflow as tf

x = tf.range(4)
np.save('x-file.npy', x)

In [28]:
# We can now read the data from the stored file back into memory.
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [29]:
# We can store a list of tensors and read them back into memory.
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [30]:
# We can even write and read a dictionary that maps from strings to tensors. 
# This is convenient when we want to read or write all the weights in a model.
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## 5.2 Loading and Saving Model Parameters

In [31]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [32]:
# Next, we store the parameters of the model as a file with the name “mlp.params”.
net.save_weights('mlp.params')

In [33]:
# To recover the model, we instantiate a clone of the original MLP model. 
# Instead of randomly initializing the model parameters, we read the parameters stored in the file directly.
clone = MLP()
clone.load_weights('mlp.params')

In [34]:
# Since both instances have the same model parameters, the computational
# result of the same input X should be the same. Let us verify this.
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

# 6. GPUs

In [40]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## 6.1 Computing Devices

In [41]:
import tensorflow as tf

tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7fedb82ba5f0>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7fedb82d8230>)

In [42]:
# We can query the number of available GPUs.
len(tf.config.experimental.list_physical_devices('GPU'))

0

In [44]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7fedb82d7a00>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7fedb82e00a0>])

## 6.2 Tensors and GPUs

In [45]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

### 6.2.1 Storage on the GPU

In [46]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [47]:
# Assuming that you have at least two GPUs, the following code will create a random tensor on the second GPU.
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.87378967, 0.48416793, 0.6413621 ],
       [0.513028  , 0.5822177 , 0.90916204]], dtype=float32)>

### 6.2.2 Copying

In [48]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [49]:
# Now that the data are on the same GPU (both Z and Y are), we can add them up.
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.8737897, 1.4841679, 1.6413621],
       [1.513028 , 1.5822177, 1.909162 ]], dtype=float32)>

In [50]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

## 6.3 Neural Networks and GPUs

In [51]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [52]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.1250172],
       [2.1250172]], dtype=float32)>

In [53]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:CPU:0')